In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
import torch
from typing import Literal, TypedDict

In [2]:
sys.path.append("..")
os.environ["USER_PATH"] = "../userdata/"

In [3]:
from system.namespace.store import get_namespace

In [4]:
namespace = get_namespace("train_db")

In [5]:
from system.embedding.store import get_embed_store
from misc.lru import LRU

In [6]:
embeds = get_embed_store(namespace)

In [7]:
role_c = "child"
role_p = "parent"

In [8]:
total_c = embeds.get_embedding_count(role_c)
total_c

689414

In [9]:
total_p = embeds.get_embedding_count(role_p)
total_p

689415

In [10]:
%%time

x_0 = np.vstack([
    embed[1].ravel().detach().numpy()
    for embed in embeds.get_all_embeddings(role_c, progress_bar=True)
])
y_0 = x_0.shape[0]
x_1 = np.vstack([
    embed[1].ravel().detach().numpy()
    for embed in embeds.get_all_embeddings(role_p, progress_bar=True)
])
y_1 = x_1.shape[0]
y = [
    "tab:blue" for _ in range(y_0)
] + [
    "tab:orange" for _ in range(y_1)
]
x = np.vstack([x_0, x_1])
x.shape

  0%|          | 0/689414 [00:00<?, ?it/s]

  0%|          | 0/689415 [00:00<?, ?it/s]

CPU times: user 4min 52s, sys: 1min 27s, total: 6min 19s
Wall time: 28min 11s


(1378829, 768)

In [11]:
x[:2, :]

array([[-0.12091923, -0.04834189,  0.05038018, ..., -0.05596649,
         0.04401225, -1.84632361],
       [-0.12091924, -0.04834188,  0.05038019, ..., -0.0559665 ,
         0.04401225, -1.84632373]])

In [12]:
def sigmoid(x):  
    return np.exp(-np.logaddexp(0, -x))

def distance(a, b):
    return np.log1p(np.exp(-np.dot(a, b)))
    # return 1.0 - sigmoid(np.dot(a, b))

In [16]:
total = total_c + total_p
# dists = np.zeros((total, total))
total * total / 1024 / 1024 / 1024

1770.6019908571616